<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%A7%D9%84%D9%83%D9%88%D9%88%D9%88%D9%88%D9%88%D9%88%D9%88%D9%88%D8%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time  # استيراد مكتبة الوقت
import gc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 84482  # أدخل رقم حسابك
password = "54q9mQ-w"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()


def free_memory():
    """تحرير الذاكرة غير المستخدمة."""
    gc.collect()  # استدعاء جمع القمامة لتحرير الذاكرة
    print("Memory has been freed.")


def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None


def calculate_atr(data, period=14):
    """Calculate Average True Range (ATR) for risk management."""
    high_low = data['high'] - data['low']
    high_close = abs(data['high'] - data['close'].shift(1))
    low_close = abs(data['low'] - data['close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr


def update_stop_loss(position, new_stop_loss):
    """تحديث وقف الخسارة للصفقة المحددة مع الحفاظ على تيك بروفت."""
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "position": position.ticket,
        "sl": new_stop_loss,
        "tp": position.tp,  # الحفاظ على تيك بروفت كما هو
        "symbol": position.symbol,
        "magic": position.magic,
        "deviation": 10,
        "comment": "Update Stop Loss",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    return result


def set_break_even_all_positions():
    """
    Sets the stop-loss to different levels based on price progress towards the target
    for all positions in the account, without moving the stop-loss if the new one is lower than the old one.
    """
    # Get all positions in the account
    all_positions = mt5.positions_get()

    if all_positions:
        for position in all_positions:
            symbol = position.symbol  # Get the symbol for the current position
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl

            # Determine the current price based on the order type
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # Calculate the distance to take profit (absolute value)
            distance_to_tp = abs(take_profit - entry_price)

            # Calculate targets (adjust for buy/sell)

            percent_target123 = entry_price + (0.10 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.10 * distance_to_tp)
            percent_target12 = entry_price + (0.20 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.20 * distance_to_tp)
            percent_target1 = entry_price + (0.30 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.30 * distance_to_tp)
            percent_target = entry_price + (0.40 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.40 * distance_to_tp)
            fifty_percent_target = entry_price + (0.50 * distance_to_tp) if position.type == 0 else entry_price - (
                        0.50 * distance_to_tp)
            fifty_target = entry_price + (0.60 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.60 * distance_to_tp)
            seventy_percent_target = entry_price + (
                    0.70 * distance_to_tp) if position.type == 0 else entry_price - (0.70 * distance_to_tp)

            seventy_five_percent_target = entry_price + (
                        0.80 * distance_to_tp) if position.type == 0 else entry_price - (0.80 * distance_to_tp)

            ninety_percent_target = entry_price + (0.90 * distance_to_tp) if position.type == 0 else entry_price - (
                        0.90 * distance_to_tp)

            new_stop_loss = current_stop_loss

            # Update stop-loss based on current price and order type

            if position.type == 0 and current_price >= percent_target123:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.60 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target123:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.60 * distance_to_tp))

            if position.type == 0 and current_price >= percent_target12:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.50 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target12:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.50 * distance_to_tp))


            if position.type == 0 and current_price >= percent_target1:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.40 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target1:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.40 * distance_to_tp))

            if position.type == 0 and current_price >= percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.03 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.03 * distance_to_tp))

            if position.type == 0 and current_price >= fifty_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.07 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.07 * distance_to_tp))

            if position.type == 0 and current_price >= fifty_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.12 * distance_to_tp))

            if position.type == 0 and current_price >= seventy_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.18 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.18 * distance_to_tp))

            if position.type == 0 and current_price >= seventy_five_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_five_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.24 * distance_to_tp))

            if position.type == 0 and current_price >= ninety_percent_target:  # Buy Order
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
            elif position.type == 1 and current_price <= ninety_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.48 * distance_to_tp))

            # Check if the new stop-loss is different and valid before updating
            if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                update_stop_loss(position, new_stop_loss)


# Call the function to set break-even for all positions
set_break_even_all_positions()


def calculate_lot_size(account_balance, risk_percentage=0.01, pip_value=10):
    """حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة."""
    risk_amount = account_balance * risk_percentage
    lot_size = risk_amount / pip_value  # يمكنك تعديل pip_value حسب الأداة المالية
    return max(0.01, lot_size)  # تأكد من أن حجم العقد لا يقل عن 0.01


def calculate_bollinger_bands(data, window=5, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)



        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data


def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        # تحقق من الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol=symbol)
        if open_positions:
            for position in open_positions:
                entry_price = position.price_open
                # تحقق من المسافة بين السعر الحالي وأي من الصفات المفتوحة
                if abs(price - entry_price) < 0.5 * atr_value:
                    print("السعر الحالي قريب جدًا من صفقة مفتوحة، لا يمكن فتح صفقة جديدة.")
                    return

        sl = price - 3 * atr_value if order_type == "buy" else price + 3 * atr_value
        tp = price + 1.5 * atr_value if order_type == "buy" else price - 1.5 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(
                f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")


def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []


def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments







def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount









def calculate_rsi(data, period=25):
    """حساب RSI ورسمه."""
    try:
        # حساب الفرق بين الأسعار
        delta = data['close'].diff()

        # حساب المكاسب والخسائر
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

        # حساب النسبة RS
        rs = gain / loss

        # حساب RSI
        data['RSI'] = 100 - (100 / (1 + rs))

        # طباعة آخر قيمة من RSI
        print(f"RSI: {data['RSI'].iloc[-1]}")

        # رسم RSI
        """plt.figure(figsize=(10, 5))
        plt.plot(data.index, data['RSI'], label='RSI', color='blue')
        plt.axhline(70, linestyle='--', alpha=0.5, color='red')  # مستوى التشبع الشرائي
        plt.axhline(30, linestyle='--', alpha=0.5, color='green')  # مستوى التشبع البيعي
        plt.title('Relative Strength Index (RSI)')
        plt.xlabel('Date')
        plt.ylabel('RSI Value')
        plt.legend()
        plt.grid()
        plt.show()"""

        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data




def calculate_stochastic(data, k_window=21, d_window=7):
    """حساب Stochastic Oscillator وإضافته إلى البيانات."""
    low_min = data['low'].rolling(window=k_window).min()
    high_max = data['high'].rolling(window=k_window).max()

    data['%K'] = 100 * (data['close'] - low_min) / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=d_window).mean()

    return data

"""def plot_stochastic(data):
    #رسم مؤشر Stochastic Oscillator.
    plt.figure(figsize=(14, 7))
    plt.plot(data.index, data['%K'], label='%K', color='blue')
    plt.plot(data.index, data['%D'], label='%D', color='red')
    plt.axhline(80, linestyle='--', alpha=0.5, color='green')
    plt.axhline(20, linestyle='--', alpha=0.5, color='red')
    plt.title('Stochastic Oscillator')
    plt.xlabel('Date')
    plt.ylabel('Stochastic Value')
    plt.legend()
    plt.show()"""


def calculate_moving_average(data, period=50):
    """حساب المتوسط المتحرك ورسمه."""
    try:
        # حساب المتوسط المتحرك
        data['MA'] = data['close'].rolling(window=period).mean()

        # طباعة آخر قيمة من المتوسط المتحرك
        print(f"Moving Average (MA): {data['MA'].iloc[-1]}")

        # رسم البيانات
        """plt.figure(figsize=(12, 6))
        plt.plot(data.index, data['close'], label='Close Price', color='blue')
        plt.plot(data.index, data['MA'], label=f'Moving Average ({period})', color='orange')
        plt.title('Close Price and Moving Average')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.grid()
        plt.show()"""

        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data



def calculate_moving_average20(data, period=20):
    """حساب المتوسط المتحرك لفترة 20."""
    try:
        data['MA20'] = data['close'].rolling(window=period).mean()
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data





def fetch_high_impact_news():
    """جمع الأخبار ذات التأثير العالي على الدولار الأمريكي من موقع Investing.com."""
    try:
        url = "https://www.investing.com/economic-calendar/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.select('tr[data-impact="high"]')  # اختيار الأخبار ذات التأثير العالي
        news_times = []

        for item in news_items:
            time_str = item.select_one('td[data-time]').get('data-time')
            news_time = datetime.fromtimestamp(int(time_str))
            news_times.append(news_time)

        return news_times
    except Exception as e:
        print(f"An error occurred while fetching high impact news: {e}")
        return []

def is_news_time(news_times):
    """التحقق مما إذا كانت هناك أخبار ذات تأثير عالي خلال ساعة قبل أو بعد الوقت الحالي."""
    current_time = datetime.now()
    for news_time in news_times:
        if news_time - timedelta(hours=1) <= current_time <= news_time + timedelta(hours=1):
            return True
    return False



def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # جلب الأخبار ذات التأثير العالي
        news_times = fetch_high_impact_news()

        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_moving_average20(data)

        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)
        data = calculate_stochastic(data)  # حساب مؤشر Stochastic
        #plot_stochastic(data)  # رسم المؤشر

        # جلب آخر صف من البيانات
        last_row = data.iloc[-1]

        # حساب عدد الصفقات المفتوحة
        open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))

        # طباعة القيم لتسهيل تتبع الأخطاء
        print(f"Last Row: {last_row}")
        print(f"Open Positions Count: {open_positions_count}")

        # فتح أمر بيع من الأسفل
        if (last_row['target'] < last_row['close'] and
            last_row['Lower_Band'] < last_row['close'] < last_row['Middle_Band'] and
            last_row['Signal_Line'] > last_row['MACD'] and
            last_row['Premium'] < last_row['Discount'] and
            50 > last_row['RSI'] > 30 and
            last_row['%K'] < last_row['%D'] and
            last_row['MA'] > last_row['MA20'] > last_row['close'] and
            open_positions_count < 5):
            result = place_order("XAUUSD", "sell", volume=0.01)
            if result is None:
                print("Failed to place sell order from below.")
            else:
                print("تم فتح أمر بيع من الأسفل.")

        # فتح أمر بيع من الأعلى
        elif (last_row['target'] > last_row['close'] and
              last_row['Middle_Band'] < last_row['close'] < last_row['Upper_Band'] and
              last_row['Signal_Line'] < last_row['MACD'] and
              last_row['Premium'] < last_row['Discount'] and
              50 > last_row['RSI'] > 30 and
              last_row['%K'] > last_row['%D'] and
              last_row['MA20'] < last_row['close'] < last_row['MA'] and
              open_positions_count < 5):
            result = place_order("XAUUSD", "sell", volume=0.01)
            if result is None:
                print("Failed to place sell order from above.")
            else:
                print("تم فتح أمر بيع من الأعلى.")

        # فتح أمر شراء من الأعلى
        elif (last_row['target'] > last_row['close'] and
              last_row['Middle_Band'] < last_row['close'] < last_row['Upper_Band'] and
              last_row['Signal_Line'] < last_row['MACD'] and
              last_row['Premium'] > last_row['Discount'] and
              50 > last_row['RSI'] < 70 and
              last_row['%K'] > last_row['%D'] and
              last_row['MA'] < last_row['MA20'] < last_row['close'] and
              open_positions_count < 5):
            result = place_order("XAUUSD", "buy", volume=0.01)
            if result is None:
                print("Failed to place buy order from above.")
            else:
                print("تم فتح أمر شراء من الأعلى.")

        # فتح أمر شراء من الأسفل
        elif (last_row['target'] < last_row['close'] and
              last_row['Lower_Band'] < last_row['close'] < last_row['Middle_Band'] and
              last_row['Signal_Line'] > last_row['MACD'] and
              last_row['Premium'] > last_row['Discount'] and
              50 > last_row['RSI'] < 70 and
              last_row['%K'] < last_row['%D'] and
              last_row['MA'] < last_row['close'] < last_row['MA20'] and
              open_positions_count < 5):
            result = place_order("XAUUSD", "buy", volume=0.01)
            if result is None:
                print("Failed to place buy order from below.")
            else:
                print("تم فتح أمر شراء من الأسفل.")

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")




def check_open_positions(symbol):
    """Check if there are any open positions for the given symbol."""
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0


# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    last_memory_cleanup = time.time()  # حفظ وقت آخر تنظيف للذاكرة
    while True:
        # جمع البيانات
        data = get_and_clean_gold_prices()

        if data is not None:
            # تحديث وقف الخسارة لجميع الصفقات
            set_break_even_all_positions()

            # بناء الاستراتيجية أو تنفيذ الأوامر
            build_strategy(data)

        # تحرير الذاكرة كل خمس ساعات
        current_time = time.time()
        if current_time - last_memory_cleanup >= 60 * 60:  # 5 ساعات
            free_memory()
            last_memory_cleanup = current_time  # تحديث وقت آخر تنظيف للذاكرة

        # تأخير زمني بين التكرارات
        time.sleep(5)  # يمكنك تعديل الوقت حسب الحاجة


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time  # استيراد مكتبة الوقت
import gc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 84482  # أدخل رقم حسابك
password = "54q9mQ-w"  # أدخل كلمة المرور
server = "RoyalInvestment-Server"  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()


def free_memory():
    """تحرير الذاكرة غير المستخدمة."""
    gc.collect()  # استدعاء جمع القمامة لتحرير الذاكرة
    print("Memory has been freed.")


def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    """Reads gold prices from MetaTrader 5 for a given symbol, cleans the data, and returns it."""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None


def calculate_atr(data, period=14):
    """Calculate Average True Range (ATR) for risk management."""
    high_low = data['high'] - data['low']
    high_close = abs(data['high'] - data['close'].shift(1))
    low_close = abs(data['low'] - data['close'].shift(1))
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr


def update_stop_loss(position, new_stop_loss):
    """تحديث وقف الخسارة للصفقة المحددة مع الحفاظ على تيك بروفت."""
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "position": position.ticket,
        "sl": new_stop_loss,
        "tp": position.tp,  # الحفاظ على تيك بروفت كما هو
        "symbol": position.symbol,
        "magic": position.magic,
        "deviation": 10,
        "comment": "Update Stop Loss",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    return result


def set_break_even_all_positions():
    """
    Sets the stop-loss to different levels based on price progress towards the target
    for all positions in the account, without moving the stop-loss if the new one is lower than the old one.
    """
    # Get all positions in the account
    all_positions = mt5.positions_get()

    if all_positions:
        for position in all_positions:
            symbol = position.symbol  # Get the symbol for the current position
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl

            # Determine the current price based on the order type
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # Calculate the distance to take profit (absolute value)
            distance_to_tp = abs(take_profit - entry_price)

            # Calculate targets (adjust for buy/sell)

            percent_target123 = entry_price + (0.10 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.10 * distance_to_tp)
            percent_target12 = entry_price + (0.20 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.20 * distance_to_tp)
            percent_target1 = entry_price + (0.30 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.30 * distance_to_tp)
            percent_target = entry_price + (0.40 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.40 * distance_to_tp)
            fifty_percent_target = entry_price + (0.50 * distance_to_tp) if position.type == 0 else entry_price - (
                        0.50 * distance_to_tp)
            fifty_target = entry_price + (0.60 * distance_to_tp) if position.type == 0 else entry_price - (
                    0.60 * distance_to_tp)
            seventy_percent_target = entry_price + (
                    0.70 * distance_to_tp) if position.type == 0 else entry_price - (0.70 * distance_to_tp)

            seventy_five_percent_target = entry_price + (
                        0.80 * distance_to_tp) if position.type == 0 else entry_price - (0.80 * distance_to_tp)

            ninety_percent_target = entry_price + (0.90 * distance_to_tp) if position.type == 0 else entry_price - (
                        0.90 * distance_to_tp)

            new_stop_loss = current_stop_loss

            # Update stop-loss based on current price and order type

            if position.type == 0 and current_price >= percent_target123:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.60 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target123:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.60 * distance_to_tp))

            if position.type == 0 and current_price >= percent_target12:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.50 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target12:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.50 * distance_to_tp))


            if position.type == 0 and current_price >= percent_target1:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (-0.40 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target1:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (-0.40 * distance_to_tp))

            if position.type == 0 and current_price >= percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.03 * distance_to_tp))
            elif position.type == 1 and current_price <= percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.03 * distance_to_tp))

            if position.type == 0 and current_price >= fifty_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.07 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.07 * distance_to_tp))

            if position.type == 0 and current_price >= fifty_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.12 * distance_to_tp))

            if position.type == 0 and current_price >= seventy_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.18 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.18 * distance_to_tp))

            if position.type == 0 and current_price >= seventy_five_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_five_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.24 * distance_to_tp))

            if position.type == 0 and current_price >= ninety_percent_target:  # Buy Order
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
            elif position.type == 1 and current_price <= ninety_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.48 * distance_to_tp))

            # Check if the new stop-loss is different and valid before updating
            if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                update_stop_loss(position, new_stop_loss)


# Call the function to set break-even for all positions
set_break_even_all_positions()


def calculate_lot_size(account_balance, risk_percentage=0.01, pip_value=10):
    """حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة."""
    risk_amount = account_balance * risk_percentage
    lot_size = risk_amount / pip_value  # يمكنك تعديل pip_value حسب الأداة المالية
    return max(0.01, lot_size)  # تأكد من أن حجم العقد لا يقل عن 0.01


def calculate_bollinger_bands(data, window=5, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)



        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data


def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        # تحقق من الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol=symbol)
        if open_positions:
            for position in open_positions:
                entry_price = position.price_open
                # تحقق من المسافة بين السعر الحالي وأي من الصفات المفتوحة
                if abs(price - entry_price) < 0.5 * atr_value:
                    print("السعر الحالي قريب جدًا من صفقة مفتوحة، لا يمكن فتح صفقة جديدة.")
                    return

        sl = price - 2 * atr_value if order_type == "buy" else price + 2 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(
                f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")




def place_order2(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        # تحقق من الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol=symbol)
        if open_positions:
            for position in open_positions:
                entry_price = position.price_open
                # تحقق من المسافة بين السعر الحالي وأي من الصفات المفتوحة
                if abs(price - entry_price) < 0.5 * atr_value:
                    print("السعر الحالي قريب جدًا من صفقة مفتوحة، لا يمكن فتح صفقة جديدة.")
                    return

        sl = price - 3 * atr_value if order_type == "buy" else price + 3 * atr_value
        tp = price + 1.5 * atr_value if order_type == "buy" else price - 1.5 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(
                f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")




def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []


def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments







def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['close'] = pd.to_numeric(data['close'], errors='coerce')
    data['low'] = pd.to_numeric(data['low'], errors='coerce')
    data['high'] = pd.to_numeric(data['high'], errors='coerce')
    data.dropna(subset=['close', 'low', 'high'], inplace=True)

    premium = data['close'] - data['low'].rolling(window=20).min()
    discount = data['high'].rolling(window=20).max() - data['close']
    return premium, discount









def calculate_rsi(data, period=25):
    """حساب RSI ورسمه."""
    try:
        # حساب الفرق بين الأسعار
        delta = data['close'].diff()

        # حساب المكاسب والخسائر
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

        # حساب النسبة RS
        rs = gain / loss

        # حساب RSI
        data['RSI'] = 100 - (100 / (1 + rs))

        # طباعة آخر قيمة من RSI
        print(f"RSI: {data['RSI'].iloc[-1]}")

        # رسم RSI
        """plt.figure(figsize=(10, 5))
        plt.plot(data.index, data['RSI'], label='RSI', color='blue')
        plt.axhline(70, linestyle='--', alpha=0.5, color='red')  # مستوى التشبع الشرائي
        plt.axhline(30, linestyle='--', alpha=0.5, color='green')  # مستوى التشبع البيعي
        plt.title('Relative Strength Index (RSI)')
        plt.xlabel('Date')
        plt.ylabel('RSI Value')
        plt.legend()
        plt.grid()
        plt.show()"""

        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data




def calculate_stochastic(data, k_window=21, d_window=7):
    """حساب Stochastic Oscillator وإضافته إلى البيانات."""
    low_min = data['low'].rolling(window=k_window).min()
    high_max = data['high'].rolling(window=k_window).max()

    data['%K'] = 100 * (data['close'] - low_min) / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=d_window).mean()

    return data

"""def plot_stochastic(data):
    #رسم مؤشر Stochastic Oscillator.
    plt.figure(figsize=(14, 7))
    plt.plot(data.index, data['%K'], label='%K', color='blue')
    plt.plot(data.index, data['%D'], label='%D', color='red')
    plt.axhline(80, linestyle='--', alpha=0.5, color='green')
    plt.axhline(20, linestyle='--', alpha=0.5, color='red')
    plt.title('Stochastic Oscillator')
    plt.xlabel('Date')
    plt.ylabel('Stochastic Value')
    plt.legend()
    plt.show()"""


def calculate_moving_average(data, period=50):
    """حساب المتوسط المتحرك ورسمه."""
    try:
        # حساب المتوسط المتحرك
        data['MA'] = data['close'].rolling(window=period).mean()

        # طباعة آخر قيمة من المتوسط المتحرك
        print(f"Moving Average (MA): {data['MA'].iloc[-1]}")

        # رسم البيانات
        """plt.figure(figsize=(12, 6))
        plt.plot(data.index, data['close'], label='Close Price', color='blue')
        plt.plot(data.index, data['MA'], label=f'Moving Average ({period})', color='orange')
        plt.title('Close Price and Moving Average')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.grid()
        plt.show()"""

        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data



def calculate_moving_average5(data, period=5):
    """حساب المتوسط المتحرك لفترة 20."""
    try:
        data['MA5'] = data['close'].rolling(window=period).mean()
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_moving_average140(data, period=140):
    """حساب المتوسط المتحرك لفترة 20."""
    try:
        data['MA140'] = data['close'].rolling(window=period).mean()
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data



def fetch_high_impact_news():
    """جمع الأخبار ذات التأثير العالي على الدولار الأمريكي من موقع Investing.com."""
    try:
        url = "https://www.investing.com/economic-calendar/"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.select('tr[data-impact="high"]')  # اختيار الأخبار ذات التأثير العالي
        news_times = []

        for item in news_items:
            time_str = item.select_one('td[data-time]').get('data-time')
            news_time = datetime.fromtimestamp(int(time_str))
            news_times.append(news_time)

        return news_times
    except Exception as e:
        print(f"An error occurred while fetching high impact news: {e}")
        return []

def is_news_time(news_times):
    """التحقق مما إذا كانت هناك أخبار ذات تأثير عالي خلال ساعة قبل أو بعد الوقت الحالي."""
    current_time = datetime.now()
    for news_time in news_times:
        if news_time - timedelta(hours=1) <= current_time <= news_time + timedelta(hours=1):
            return True
    return False



def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # جلب الأخبار ذات التأثير العالي
        news_times = fetch_high_impact_news()

        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_moving_average5(data)
        data = calculate_moving_average140(data)
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)
        data = calculate_stochastic(data)  # حساب مؤشر Stochastic
        #plot_stochastic(data)  # رسم المؤشر

        # جلب آخر صف من البيانات
        last_row = data.iloc[-1]

        # حساب عدد الصفقات المفتوحة
        open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))

        # طباعة القيم لتسهيل تتبع الأخطاء
        print(f"Last Row: {last_row}")
        print(f"Open Positions Count: {open_positions_count}")

        # فتح أمر بيع من الأسفل
        if (last_row['target'] < last_row['close'] and

            last_row['MA140']> last_row['MA'] > last_row['MA5'] > last_row['close'] and
            open_positions_count < 5):
            result = place_order("XAUUSD", "sell", volume=0.01)
            if result is None:
                print("Failed to place sell order from below.")
            else:
                print("تم فتح أمر بيع من الأسفل.")

        # فتح أمر بيع من الأعلى
        elif (last_row['target'] > last_row['close'] and

              last_row['MA5'] < last_row['close'] < last_row['MA'] < last_row['MA140']and
              open_positions_count < 5):
            result = place_order2("XAUUSD", "sell", volume=0.01)
            if result is None:
                print("Failed to place sell order from above.")
            else:
                print("تم فتح أمر بيع من الأعلى.")

        # فتح أمر شراء من الأعلى
        elif (last_row['target'] > last_row['close'] and

              last_row['MA140']< last_row['MA'] < last_row['MA5'] < last_row['close'] and
              open_positions_count < 5):
            result = place_order("XAUUSD", "buy", volume=0.01)
            if result is None:
                print("Failed to place buy order from above.")
            else:
                print("تم فتح أمر شراء من الأعلى.")

        # فتح أمر شراء من الأسفل
        elif (last_row['target'] < last_row['close'] and

              last_row['MA140'] < last_row['MA'] < last_row['close'] < last_row['MA5'] and
              open_positions_count < 5):
            result = place_order2("XAUUSD", "buy", volume=0.01)
            if result is None:
                print("Failed to place buy order from below.")
            else:
                print("تم فتح أمر شراء من الأسفل.")

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")




def check_open_positions(symbol):
    """Check if there are any open positions for the given symbol."""
    positions = mt5.positions_get(symbol=symbol)
    return len(positions) > 0


# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    last_memory_cleanup = time.time()  # حفظ وقت آخر تنظيف للذاكرة
    while True:
        # جمع البيانات
        data = get_and_clean_gold_prices()

        if data is not None:
            # تحديث وقف الخسارة لجميع الصفقات
            set_break_even_all_positions()

            # بناء الاستراتيجية أو تنفيذ الأوامر
            build_strategy(data)

        # تحرير الذاكرة كل خمس ساعات
        current_time = time.time()
        if current_time - last_memory_cleanup >= 60 * 60:  # 5 ساعات
            free_memory()
            last_memory_cleanup = current_time  # تحديث وقت آخر تنظيف للذاكرة

        # تأخير زمني بين التكرارات
        time.sleep(5)  # يمكنك تعديل الوقت حسب الحاجة
